### Praktikum D1 — Inisialisasi Kamera dan Akuisisi Citra 

Praktikum ini merupakan tahap awal dalam mengenal sistem visi komputer. Mahasiswa akan  belajar membuka perangkat kamera, menampilkan citra secara berkelanjutan, serta  memastikan bahwa perangkat keras dan perangkat lunak dapat berkomunikasi dengan baik.  Proses ini menjadi fondasi bagi semua eksperimen berikutnya yang menggunakan aliran video  sebagai input utama

In [2]:
import cv2, time 

cap = cv2.VideoCapture(0) 

if not cap.isOpened(): 
    raise RuntimeError("Kamera tidak bisa dibuka. Coba index 1/2.") 

frames, t0 = 0, time.time() 

while True: 
    ok, frame = cap.read() 
    if not ok: 
        break 
    frames += 1 

    if time.time() - t0 >= 1.0: 
        cv2.setWindowTitle("Preview", f"Preview (FPS ~ {frames})") 
        frames, t0 = 0, time.time() 
    cv2.imshow("Preview", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break 

cap.release() 
cv2.destroyAllWindows()


### Praktikum D2 — Deteksi Pose dan Analisis Sudut 

MediaPipe Pose menyediakan 33 landmark tubuh yang dapat digunakan untuk menganalisis  posisi dan orientasi manusia. Dalam praktikum ini, mahasiswa akan mengimplementasikan  algoritma deteksi pose dan menghitung sudut sendi berdasarkan koordinat tiga titik utama  tubuh menggunakan prinsip geometri vektor

In [4]:
import cv2, numpy as np
import mediapipe
from cvzone.PoseModule import PoseDetector
print(mediapipe.__version__)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(staticMode=False, modelComplexity=1,
                        enableSegmentation=False, detectionCon=0.5,
                        trackCon=0.5)

while True:
    # Tangkap setiap frame dari webcam
    success, img = cap.read()
    if not success: break

    # Temukan pose manusia dalam frame
    img = detector.findPose(img)

    # Temukan landmark, bounding box, dan pusat tubuh dalam frame
    # Set draw=True untuk menggambar landmark dan bounding box pada gambar
    lmList, bboxInfo = detector.findPosition(img, draw=True,
                                             bboxWithHands=False)

    # Periksa apakah ada landmark tubuh yang terdeteksi
    if lmList:
        # Dapatkan pusat bounding box di sekitar tubuh
        center = bboxInfo["center"]
        # Gambar lingkaran di pusat bounding box
        cv2.circle(img, center, 5, (255, 0, 255), cv2.FILLED)

        # Hitung jarak antara landmark 11 dan 15 dan gambarkan pada gambar
        length, img, info = detector.findDistance(lmList[11][0:2],
                                                  lmList[15][0:2],
                                                  img=img,
                                                  color=(255, 0, 0),
                                                  scale=10)

        # Hitung sudut antara landmark 11, 13, dan 15 dan gambarkan pada gambar
        angle, img = detector.findAngle(lmList[11][0:2],
                                        lmList[13][0:2],
                                        lmList[15][0:2],
                                        img=img,
                                        color=(0, 0, 255),
                                        scale=10)

        # Periksa apakah sudut mendekati 50 derajat dengan offset 10
        isCloseAngle50 = detector.angleCheck(myAngle=angle,
                                             targetAngle=50,
                                             offset=10)

        # Cetak hasil pemeriksaan sudut
        print(isCloseAngle50)

    cv2.imshow("Pose + Angle", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

0.10.20
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fa

### Praktikum D3 — Deteksi Wajah dan Analisis Kedipan Mata 

Face Mesh memetakan 468 titik pada wajah, termasuk titik di sekitar mata. Praktikum ini  memperkenalkan konsep Eye Aspect Ratio (EAR), yaitu rasio antara jarak vertikal dan  horizontal mata yang digunakan untuk menentukan apakah mata dalam kondisi terbuka atau  tertutup.

In [7]:
import cv2, numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector

# Indeks landmark mata kiri yang digunakan (MediaPipe face mesh indices)
L_TOP, L_BOTTOM, L_LEFT, L_RIGHT = 159, 145, 33, 133 

# Fungsi untuk menghitung jarak Euclidean antara dua titik (p1 dan p2)
def dist(p1, p2): 
    return np.linalg.norm(np.array(p1) - np.array(p2)) 

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

# Inisialisasi objek FaceMeshDetector
detector = FaceMeshDetector(staticMode=False, maxFaces=2,
                            minDetectionCon=0.5, minTrackCon=0.5)

# Variabel untuk menghitung kedipan sederhana
blink_count = 0 
closed_frames = 0 
CLOSED_FRAMES_THRESHOLD = 3 # Jumlah frame berturut-turut untuk dianggap kedipan 
EYE_AR_THRESHOLD = 0.20 # Ambang batas Eye Aspect Ratio (EAR) untuk menilai mata tertutup 
is_closed = False 

while True:
    ok, img = cap.read()
    if not ok: break
    
    # Temukan Face Mesh di dalam frame
    img, faces = detector.findFaceMesh(img, draw=True) 
    
    if faces: 
        face = faces[0] # Ambil data untuk wajah pertama (list of 468 (x,y) landmarks)
        
        # Hitung jarak vertikal dan horizontal mata kiri
        v = dist(face[L_TOP], face[L_BOTTOM]) 
        h = dist(face[L_LEFT], face[L_RIGHT]) 
        
        # Hitung Eye Aspect Ratio (EAR)
        # 1e-8 ditambahkan untuk menghindari pembagian dengan nol
        ear = v / (h + 1e-8) 
        
        # Tampilkan nilai EAR pada frame
        cv2.putText(img, f"EAR(L): {ear:.3f}", (20,40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2) 
        
        # Logika counter kedipan sederhana:
        if ear < EYE_AR_THRESHOLD: 
            closed_frames += 1 
            
            # Jika mata tertutup cukup lama DAN belum dicatat sebagai kedipan baru
            if closed_frames >= CLOSED_FRAMES_THRESHOLD and not is_closed: 
                blink_count += 1 
                is_closed = True 
        else: 
            # Jika mata terbuka, reset penghitung frame tertutup
            closed_frames = 0 
            is_closed = False 

        # Tampilkan jumlah kedipan pada frame
        cv2.putText(img, f"Blink: {blink_count}", (20,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2) 
        
    cv2.imshow("FaceMesh + EAR", img) 
    
    # Keluar jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'): break 

cap.release()
cv2.destroyAllWindows()

### Praktikum D4 — Deteksi Tangan dan Penghitungan Jumlah Jari 

MediaPipe Hands mendeteksi 21 landmark pada setiap tangan. Berdasarkan hubungan antara  ujung jari (tip) dan sendi bawahnya (PIP), dapat ditentukan apakah jari tersebut dalam posisi  terangkat atau tidak. Pendekatan ini digunakan untuk menghitung jumlah jari terbuka. 

In [9]:
import cv2
from cvzone.HandTrackingModule import HandDetector

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

# Inisialisasi HandDetector
detector = HandDetector(staticMode=False, maxHands=1, modelComplexity=1,
                        detectionCon=0.5, minTrackCon=0.5)

while True:
    ok, img = cap.read()
    if not ok: break
    
    # Temukan tangan dalam frame. flipType=True memirorkan tampilan UI.
    hands, img = detector.findHands(img, draw=True, flipType=True) 
    
    if hands: 
        hand = hands[0] # Ambil data tangan pertama
        
        # Cek jari mana yang terangkat (list [0/1] sepanjang 5)
        fingers = detector.fingersUp(hand) 
        
        # Hitung jumlah jari yang terangkat
        count = sum(fingers) 
        
        # Tampilkan jumlah jari dan status jari pada frame
        cv2.putText(img, f"Fingers: {count} {fingers}", (20,40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2) 
    
    cv2.imshow("Hands + Fingers", img) 
    
    # Keluar dari loop jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'): break 

cap.release() 
cv2.destroyAllWindows()

### Praktikum D5 — Pengenalan Gestur Tangan (Hand Gesture Recognition) 
Gestur tangan merupakan salah satu bentuk komunikasi nonverbal yang dapat diterjemahkan  oleh sistem visi komputer. Dalam praktikum ini, mahasiswa akan merancang aturan  geometris untuk mengenali gestur dasar seperti “Thumbs Up”, “OK”, dan “Rock–Paper– Scissors”. 


In [10]:
import cv2, numpy as np
from cvzone.HandTrackingModule import HandDetector

# Fungsi untuk menghitung jarak Euclidean antara dua titik (a dan b)
def dist(a, b): 
    return np.linalg.norm(np.array(a) - np.array(b)) 

def classify_gesture(hand):
    # Dapatkan 21 landmark (x,y,z) tangan
    lm = hand["lmList"] 
    if not lm:
        return "NO_HAND"
        
    wrist = np.array(lm[0][:2]) 
    thumb_tip = np.array(lm[4][:2]) 
    index_tip = np.array(lm[8][:2]) 
    middle_tip = np.array(lm[12][:2]) 
    ring_tip = np.array(lm[16][:2]) 
    pinky_tip = np.array(lm[20][:2]) 
    
    # Hitung rata-rata jarak dari ujung jari ke pergelangan tangan (wrist)
    # Ini membantu membedakan CLOSED (ROCK) dari OPEN (PAPER)
    r_mean = np.mean([dist(index_tip, wrist), dist(middle_tip, wrist),
                      dist(ring_tip, wrist), dist(pinky_tip, wrist), 
                      dist(thumb_tip, wrist)]) 
    
    # --- Aturan Klasifikasi ---
    
    # 1. OK (Jari telunjuk dan ibu jari bertemu)
    if dist(thumb_tip, index_tip) < 35: 
        return "OK" 
    
    # 2. THUMBS_UP (Ibu jari tinggi dan jauh dari pergelangan tangan)
    # Perlu akses ke fingersUp untuk validasi yang lebih baik (misalnya, jari lain tertutup)
    if (thumb_tip[1] < wrist[1] - 40) and (dist(thumb_tip, wrist) > 0.8 * dist(index_tip, wrist)): 
        return "THUMBS_UP" 
    
    # 3. ROCK (Tangan terkepal) - Jarak rata-rata pendek
    if r_mean < 120: 
        return "ROCK" 
    
    # 4. PAPER (Tangan terbuka) - Jarak rata-rata panjang
    if r_mean > 200: 
        return "PAPER" 
    
    # 5. V/PEACE (Jari telunjuk dan tengah terbuka, yang lain tertutup)
    # Untuk validasi ini, kita memerlukan hasil dari fingersUp.
    # Kita asumsikan lmList[16] adalah landmark tip jari.
    # Namun, karena bagian ini terpotong di input Anda, 
    # kita akan lengkapi dengan asumsi dasar:
    
    # JIKA Kode Lanjutan Anda Mencoba Mendeteksi V/PEACE:
    # if dist(index_tip, wrist) > 180 and dist(middle_tip, wrist) > 180 and ... 
    
    # Asumsi default jika tidak ada aturan yang cocok
    return "UNKNOWN"


cap = cv2.VideoCapture(0)
if not cap.isOpened(): 
    raise RuntimeError("Kamera tidak bisa dibuka.") 

# Inisialisasi HandDetector: maxHands=1 untuk mendeteksi satu tangan saja
detector = HandDetector(staticMode=False, maxHands=1, modelComplexity=1, 
                        detectionCon=0.5, minTrackCon=0.5) 

while True:
    ok, img = cap.read() 
    if not ok: break 
    
    # Temukan tangan. flipType=True disarankan untuk tampilan cermin.
    hands, img = detector.findHands(img, draw=True, flipType=True) 
    
    gesture = "Mencari Tangan..."
    
    if hands: 
        hand = hands[0] # Data tangan pertama
        
        # Klasifikasikan gerakan tangan
        gesture = classify_gesture(hand)

    # Tampilkan hasil klasifikasi gerakan
    cv2.putText(img, f"Gesture: {gesture}", (20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2) 
    
    cv2.imshow("Gesture Recognition", img) 
    
    if cv2.waitKey(1) & 0xFF == ord('q'): break 

cap.release() 
cv2.destroyAllWindows()

### Praktikum D6 — Analisis Gerakan Tubuh dan Penghitung Aktivitas 
Praktikum ini mengintegrasikan kemampuan deteksi pose dengan logika analisis gerakan.  Mahasiswa akan menerapkan penghitungan sudut dan rasio jarak untuk mengenali dua posisi  utama (naik dan turun), serta menerapkan teknik debounce untuk menghindari penghitungan  ganda. 


In [17]:
import cv2, numpy as np 
from collections import deque
from cvzone.PoseModule import PoseDetector 
# Import time untuk membuat nama file yang unik (opsional)
import time 

# --- Konfigurasi ---
MODE = "squat" 
KNEE_DOWN, KNEE_UP = 120, 180 
DOWN_R, UP_R = 0.85, 1.00 
SAMPLE_OK = 4 

cap = cv2.VideoCapture(0) 
if not cap.isOpened(): 
    raise RuntimeError("Kamera tidak bisa dibuka.") 

# 1. MENDAPATKAN PROPERTI VIDEO & INISIALISASI VIDEO WRITER
FRAME_WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FRAME_HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS = cap.get(cv2.CAP_PROP_FPS)
if FPS == 0:
    FPS = 30 

OUTPUT_FILENAME = f'pose_counter__{time.strftime("%Y%m%d_%H%M%S")}.mp4'
FOURCC = cv2.VideoWriter_fourcc(*'mp4v') # Codec untuk file .mp4
out = cv2.VideoWriter(OUTPUT_FILENAME, FOURCC, FPS, (FRAME_WIDTH, FRAME_HEIGHT))

print(f"Mulai merekam video ke file: {OUTPUT_FILENAME}")
print(f"Resolusi: {FRAME_WIDTH}x{FRAME_HEIGHT}, FPS: {FPS:.2f}")

# Inisialisasi PoseDetector
detector = PoseDetector(staticMode=False, modelComplexity=1, 
                        enableSegmentation=False, detectionCon=0.5, trackCon=0.5) 

count, state = 0, "up" 
debounce = deque(maxlen=6) 

# Fungsi untuk menghitung rasio push-up
def ratio_pushup(lm): 
    sh = np.array(lm[11][1:3]) # Shoulder 
    wr = np.array(lm[15][1:3]) # Wrist 
    hp = np.array(lm[23][1:3]) # Hip 
    return np.linalg.norm(sh - wr) / (np.linalg.norm(sh - hp) + 1e-8) 

while True: 
    ok, img = cap.read()
    if not ok: break 
    
    img = detector.findPose(img, draw=True) 
    lmList, _ = detector.findPosition(img, draw=False) 
    flag = None 
    
    if lmList: 
        if MODE == "squat":
            # Logika Squat
            angL, img = detector.findAngle(lmList[23][0:2], lmList[25][0:2], lmList[27][0:2], 
                                           img=img, color=(0, 0, 255), scale=10) 
            angR, img = detector.findAngle(lmList[24][0:2], lmList[26][0:2], lmList[28][0:2], 
                                           img=img, color=(0, 255, 0), scale=10) 
            ang = (angL + angR) / 2.0 
            
            if ang < KNEE_DOWN: 
                flag = "down" 
            elif ang > KNEE_UP: 
                flag = "up" 
                
            cv2.putText(img, f"Knee: {ang:5.1f}", (20,70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2) 
                        
        else: 
            # Logika Push-up
            r = ratio_pushup(lmList) 
            
            if r < DOWN_R: 
                flag = "down" 
            elif r > UP_R: 
                flag = "up" 
                
            cv2.putText(img, f"Ratio: {r:4.2f}", (20,70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2) 
        
        # Debounce dan Counter Logika
        debounce.append(flag) 
        
        if debounce.count("down") >= SAMPLE_OK and state == "up": 
            state = "down"
            
        if debounce.count("up") >= SAMPLE_OK and state == "down":
            state = "up"
            count += 1 
            
    # Tampilan UI
    cv2.putText(img, f"Mode: {MODE.upper()} Count: {count}", (20,40), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2) 
    cv2.putText(img, f"State: {state}", (20,100), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2) 
    
    # 2. TULIS FRAME KE FILE VIDEO
    out.write(img)

    cv2.imshow("Pose Counter", img) 
    
    # Input Kontrol
    key = cv2.waitKey(1) & 0xFF 
    if key == ord('q'): break 
    
    if key == ord('m'): 
        MODE = "pushup" if MODE == "squat" else "squat"
        count, state = 0, "up"
        debounce.clear()

# 3. PENUTUP
cap.release() 
out.release() # Penting: Tutup VideoWriter
cv2.destroyAllWindows()
print("Selesai merekam dan menutup semua jendela.")

Mulai merekam video ke file: pose_counter__20251207_100806.mp4
Resolusi: 640x480, FPS: 30.00
Selesai merekam dan menutup semua jendela.
